### Importacion de librerias

In [9]:
import pandas as pd
import numpy as np

### Carga de reportes

In [10]:
ventas_cerradas = pd.read_csv('current_reports/notas_cerradas.csv')
ventas_canceladas = pd.read_csv('current_reports/notas_canceladas.csv')
clientes = pd.read_csv('current_reports/clientes.csv')
df_ventas_cerradas = pd.DataFrame(ventas_cerradas)
df_ventas_canceladas = pd.DataFrame(ventas_canceladas)
df_clientes = pd.DataFrame(clientes)

### Manipulacion reporte de clientes

In [11]:
df_clientes.rename(columns={'index': 'Client'}, inplace=True)
df_clientes = df_clientes[["Client","seller"]]
df_clientes.set_index('Client', inplace=True)
df_clientes['seller'] = df_clientes['seller'].map(lambda x: False if (x == "-") else x) 

### Manipulacion de reporte de notas canceladas

In [12]:
df_ventas_canceladas.rename(columns={'index.1': 'Client'}, inplace=True)
df_ventas_canceladas = df_ventas_canceladas[["Código", "Client", "pos_client", "Sucursal", "canceled_on", "canceled_by", "Cobrada por", "web", "piezas",
                                        "Tipo de venta", "total", "Descuento", "Efectivo", "tr0v1", "Tarjetas", "transferencias", "depósitos"]]
df_ventas_canceladas['Código'] = df_ventas_canceladas['Código'].astype('string')
df_ventas_canceladas['BN-Cliente'] = df_ventas_canceladas['Client'].astype('int64')
df_ventas_canceladas['POS-Cliente'] = df_ventas_canceladas['pos_client'].astype('string')
df_ventas_canceladas['# Piezas'] = df_ventas_canceladas['piezas'].astype('string')
df_ventas_canceladas['Sucursal'] = df_ventas_canceladas['Sucursal'].astype('string')
df_ventas_canceladas['Cancelada por'] = df_ventas_canceladas['canceled_by'].astype('string')
df_ventas_canceladas['F/h Cancelacion'] = df_ventas_canceladas['canceled_on'].astype('string')
df_ventas_canceladas['Cobrada por'] = df_ventas_canceladas['Cobrada por'].astype('string')
df_ventas_canceladas['Pedido Web'] = df_ventas_canceladas['web'].astype('bool')
df_ventas_canceladas['Total Nota'] = df_ventas_canceladas['total']
df_ventas_canceladas['Tipo de venta'] = df_ventas_canceladas['Tipo de venta'].astype('string')
df_ventas_canceladas['Descuento Esp.'] = df_ventas_canceladas['Descuento'].map(lambda x: "Sin Descuento" if ( x == "<NULL>" ) else x).astype('string')
df_ventas_canceladas['Efectivo'] = df_ventas_canceladas['Efectivo'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_canceladas['Tr0v1'] = df_ventas_canceladas['tr0v1'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_canceladas['Tarjetas'] = df_ventas_canceladas['Tarjetas'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_canceladas['Transferencias'] = df_ventas_canceladas['transferencias'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_canceladas['Depósitos'] = df_ventas_canceladas['depósitos'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_canceladas['Total Venta'] = df_ventas_canceladas['Efectivo'] + df_ventas_canceladas['Tr0v1'] + df_ventas_canceladas['Tarjetas'] + df_ventas_canceladas['Transferencias'] + df_ventas_canceladas['Depósitos']
df_ventas_canceladas['Devolucion'] = df_ventas_canceladas['total'] - df_ventas_canceladas['Total Venta']

# Union tabla clientes
df_ventas_canceladas = df_ventas_canceladas.merge(df_clientes, how='left', on='Client' )
df_ventas_canceladas['Vendedor Asignado'] = df_ventas_canceladas['seller']

# Asiganando notas para comision segun reglas de negocio
df_ventas_canceladas['Comisionado por'] = np.where(df_ventas_canceladas['seller'] == False, df_ventas_canceladas['Cobrada por'], df_ventas_canceladas['seller'])

# Ordenando Columnas
df_ventas_canceladas = df_ventas_canceladas[[
    'Código', 'BN-Cliente', 'POS-Cliente', 'Sucursal', 'F/h Cancelacion', 'Cancelada por', 'Pedido Web', 'Tipo de venta', 'Descuento Esp.', '# Piezas', 
    'Devolucion', 'Efectivo', 'Tr0v1', 'Tarjetas', 'Depósitos', 'Transferencias', 'Total Nota', 'Total Venta', 'Vendedor Asignado', 'Comisionado por'
    ]]

# Formato reporte de notas de credito
with pd.ExcelWriter('new_reports/cancelaciones.xlsx') as writer:
    df_ventas_canceladas.to_excel(writer, index=False)

### Manipulación de reporte notas cerradas

In [13]:
df_ventas_cerradas.rename(columns={'index': 'Client'}, inplace=True)
df_ventas_cerradas = df_ventas_cerradas[["Código", "Client", "pos_client", "Sucursal", "F/h Cierre", "Cobrada por", "web", "piezas",
                                        "Tipo de venta", "total", "Descuento", "Efectivo", "tr0v1", "Tarjetas", "transferencias", "depósitos"]]
df_ventas_cerradas['Código'] = df_ventas_cerradas['Código'].astype('string')
df_ventas_cerradas['BN-Cliente'] = df_ventas_cerradas['Client'].astype('int64')
df_ventas_cerradas['POS-Cliente'] = df_ventas_cerradas['pos_client'].astype('string')
df_ventas_cerradas['# Piezas'] = df_ventas_cerradas['piezas'].astype('string')
df_ventas_cerradas['Sucursal'] = df_ventas_cerradas['Sucursal'].astype('string')
df_ventas_cerradas['Cobrada por'] = df_ventas_cerradas['Cobrada por'].astype('string')
df_ventas_cerradas['Pedido Web'] = df_ventas_cerradas['web'].astype('bool')
df_ventas_cerradas['Total Nota'] = df_ventas_cerradas['total']
df_ventas_cerradas['Tipo de venta'] = df_ventas_cerradas['Tipo de venta'].astype('string')
df_ventas_cerradas['Descuento Esp.'] = df_ventas_cerradas['Descuento'].map(lambda x: "Sin Descuento" if ( x == "<NULL>" ) else x).astype('string')
df_ventas_cerradas['Efectivo'] = df_ventas_cerradas['Efectivo'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_cerradas['Tr0v1'] = df_ventas_cerradas['tr0v1'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_cerradas['Tarjetas'] = df_ventas_cerradas['Tarjetas'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_cerradas['Transferencias'] = df_ventas_cerradas['transferencias'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_cerradas['Depósitos'] = df_ventas_cerradas['depósitos'].map(lambda x: 0 if ( x == "<NULL>" ) else x).astype('float64')
df_ventas_cerradas['Total Venta'] = df_ventas_cerradas['Efectivo'] + df_ventas_cerradas['Tr0v1'] + df_ventas_cerradas['Tarjetas'] + df_ventas_cerradas['Transferencias'] + df_ventas_cerradas['Depósitos']
df_ventas_cerradas['Devolucion'] = df_ventas_cerradas['total'] - df_ventas_cerradas['Total Venta']

# Union tabla clientes
df_ventas_cerradas = df_ventas_cerradas.merge(df_clientes, how='left', on='Client')
df_ventas_cerradas['Vendedor Asignado'] = df_ventas_cerradas['seller']


# Asiganando notas para comision segun reglas de negocio
df_ventas_cerradas['Comisionado por'] = np.where(df_ventas_cerradas['seller'] == False, df_ventas_cerradas['Cobrada por'], df_ventas_cerradas['seller'])

# Ordenando Columnas
df_ventas_cerradas = df_ventas_cerradas[[
    'Código', 'BN-Cliente', 'POS-Cliente', 'Sucursal', 'F/h Cierre', 'Pedido Web', 'Tipo de venta', 'Descuento Esp.', '# Piezas', 
    'Devolucion', 'Efectivo', 'Tr0v1', 'Tarjetas', 'Depósitos', 'Transferencias', 'Total Nota', 'Total Venta', 'Cobrada por', 'Vendedor Asignado', 'Comisionado por'
    ]]

# Formato reporte de notas de credito
with pd.ExcelWriter('new_reports/ventas.xlsx') as writer:
    df_ventas_cerradas.to_excel(writer, index=False)

### Tablas Dinamicas ventas y cancelaciones.

In [14]:
venta_x_vendedor = pd.pivot_table(df_ventas_cerradas, index='Comisionado por', values=['Total Venta', "Devolucion"], aggfunc=[np.mean, np.sum])
venta_x_vendedor['Ventas'] = venta_x_vendedor['sum']['Total Venta']
venta_x_vendedor = venta_x_vendedor.sort_values('Ventas', ascending=False)
venta_x_vendedor['Total Venta'] = venta_x_vendedor['sum']['Total Venta']
venta_x_vendedor['Total Devoluciones Ventas'] = venta_x_vendedor['sum']['Devolucion']
venta_x_vendedor['Nota Promedio'] = venta_x_vendedor['mean']['Total Venta'].map(lambda x: np.round(x, decimals=1))
venta_x_vendedor = venta_x_vendedor[['Nota Promedio', 'Total Venta', 'Total Devoluciones Ventas']]

In [15]:
cancelaciones_x_vendedor = pd.pivot_table(df_ventas_canceladas, index='Comisionado por', values=['Total Venta', "Devolucion"], aggfunc=[np.mean, np.sum])
cancelaciones_x_vendedor['Cancelaciones'] = cancelaciones_x_vendedor['sum']['Total Venta']
cancelaciones_x_vendedor = cancelaciones_x_vendedor.sort_values('Cancelaciones', ascending=False)
cancelaciones_x_vendedor['Total de Cancelaciones'] = cancelaciones_x_vendedor['sum']['Total Venta']
cancelaciones_x_vendedor['Total Devoluciones Canceladas'] = cancelaciones_x_vendedor['sum']['Devolucion']
cancelaciones_x_vendedor = cancelaciones_x_vendedor[['Total de Cancelaciones' ,'Total Devoluciones Canceladas']]

## Resumen de ventas, cancelaciones y devoluciones.

In [16]:
# Union de tablas Ventas y Cancelaciones
def formatPesos(x):
    return "$ {:.1f}".format(x)
def formatPorcentaje(x):
    return "{:.1f} %".format(x*100)
ventas_a_comisionar = venta_x_vendedor.merge(cancelaciones_x_vendedor, how="outer", on='Comisionado por' )
ventas_a_comisionar = ventas_a_comisionar.fillna(0)
ventas_a_comisionar["Subtotal Venta"] = ventas_a_comisionar["Total Venta"] - ventas_a_comisionar["Total de Cancelaciones"]
ventas_a_comisionar["% Devoluciones"] = (ventas_a_comisionar["Total Devoluciones Ventas"] - ventas_a_comisionar["Total Devoluciones Canceladas"])
ventas_a_comisionar["% Devoluciones"] = (ventas_a_comisionar["% Devoluciones"] / ventas_a_comisionar["Subtotal Venta"]).apply(formatPorcentaje)
ventas_a_comisionar["Nota Promedio"] = ventas_a_comisionar["Nota Promedio"].apply(formatPesos)
ventas_a_comisionar["Total Venta"] = ventas_a_comisionar["Total Venta"].apply(formatPesos)
ventas_a_comisionar["Total de Cancelaciones"] = ventas_a_comisionar["Total de Cancelaciones"].apply(formatPesos)
ventas_a_comisionar["Subtotal Venta"] = ventas_a_comisionar["Subtotal Venta"].apply(formatPesos)
ventas_a_comisionar[['Nota Promedio', 'Total Venta', 'Total de Cancelaciones', 'Subtotal Venta', '% Devoluciones']]

,Nota Promedio,Total Venta,Total de Cancelaciones,Subtotal Venta,% Devoluciones
,,,,,
Comisionado por,,,,,
Brian Ulises,$ 1131.6,$ 89392.5,$ 35.0,$ 89357.5,5.5 %
Alejandra,$ 741.4,$ 38551.4,$ 0.0,$ 38551.4,5.4 %
Mariana,$ 316.6,$ 37676.0,$ 0.0,$ 37676.0,0.0 %
Karol Viridiana,$ 687.5,$ 34373.1,$ 6474.0,$ 27899.1,0.0 %
Pamela,$ 1130.8,$ 33923.6,$ 0.0,$ 33923.6,0.7 %
Bryan,$ 633.6,$ 27880.1,$ 0.0,$ 27880.1,5.1 %
Patricia,$ 859.7,$ 26649.4,$ 2408.0,$ 24241.4,1.4 %
Diana Aidee,$ 739.7,$ 25888.7,$ 1331.5,$ 24557.2,1.4 %
